In [1]:
import numpy as np
import pandas as pd
import pickle
from keras.models import Model
from keras.layers import Dense, Embedding, Input, PReLU, Flatten
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
max_features = 20000
maxlen = 100

In [3]:
DATA_PATH = "../data/"
train = pd.read_csv(DATA_PATH + "train.csv")
test = pd.read_csv(DATA_PATH + "test.csv")
train = train.sample(frac=1)
list_sentences_train = train["comment_text"].fillna("CVxTz").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").values

In [4]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))

In [5]:
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

In [6]:
X_train.shape

(159571, 100)

In [7]:
y.shape

(159571, 6)

In [27]:
sess = tf.Session()
init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init)

In [8]:
import tensorflow as tf


def f2_score(y_true, y_pred):
    y_true = tf.cast(y_true, "int32")
    y_pred = tf.cast(tf.round(y_pred), "int32") # implicit 0.5 threshold via tf.round
    y_correct = y_true * y_pred
    sum_true = tf.reduce_sum(y_true, axis=1)
    sum_pred = tf.reduce_sum(y_pred, axis=1)
    sum_correct = tf.reduce_sum(y_correct, axis=1)
    precision = sum_correct / sum_pred
    recall = sum_correct / sum_true
    f_score = 5 * precision * recall / (4 * precision + recall)
    f_score = tf.where(tf.is_nan(f_score), tf.zeros_like(f_score), f_score)
    return tf.reduce_mean(f_score)

In [17]:
def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.contrib.metrics.streaming_auc(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

In [18]:
from sklearn.metrics import roc_auc_score

def mean_roc_auc(y_true, y_pred):
    y_true = tf.cast(y_true, "int32")
    y_pred = tf.cast(tf.round(y_pred), "int32") # implicit 0.5 threshold via tf.round
    roc_aucs = []
    for i in range(0, y_pred.shape[1]):
        roc_aucs.append(auc_roc(y_pred[:, i], y_true[:, i]))
    return tf.reduce_mean(roc_aucs)

In [19]:
def get_model():
    embed_size = 100
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Flatten()(x)
    x = Dense(50, activation = "relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(50, activation = "relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=[mean_roc_auc])

    return model

In [29]:
model = get_model()
batch_size = 10
epochs = 5

In [30]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 100, 100)          2000000   
_________________________________________________________________
flatten_3 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 50)                500050    
_________________________________________________________________
dropout_5 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_6 (Dropout)          (None, 50)                0         
__________

In [22]:
MODEL_PATH = "../models/"

In [23]:
model_name = "deep_dense_model_mean_roc_auc.hdf5"

In [24]:
file_path = MODEL_PATH + model_name
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=3)
callbacks_list = [checkpoint, early] #early

In [31]:
model.fit(X_train, y, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=callbacks_list)
# model.load_weights(file_path)

Train on 127656 samples, validate on 31915 samples
Epoch 1/5


FailedPreconditionError: Attempting to use uninitialized value metrics_2/mean_roc_auc/auc/true_positives
	 [[Node: metrics_2/mean_roc_auc/auc/true_positives/read = Identity[T=DT_FLOAT, _class=["loc:@metrics_2/mean_roc_auc/auc/true_positives"], _device="/job:localhost/replica:0/task:0/gpu:0"](metrics_2/mean_roc_auc/auc/true_positives)]]
	 [[Node: metrics_2/mean_roc_auc/auc_5/Cast/_435 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_769_metrics_2/mean_roc_auc/auc_5/Cast", tensor_type=DT_BOOL, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'metrics_2/mean_roc_auc/auc/true_positives/read', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykern

In [ ]:
model.evaluate(X_train, y)

In [ ]:
y_test = model.predict(X_test)

In [ ]:
y_test_rounded = np.round(y_test)

In [ ]:
sample_submission = pd.read_csv("../submissions/sample_submission.csv")
sample_submission[list_classes] = y_test_rounded
sample_submission.to_csv("../submissions/" + model_name + ".csv", index=False)

In [ ]:
y_train = model.predict(X_train)

In [ ]:
pickle.dump(y_train, open("../submissions/TRAIN" + model_name + ".p", "wb"))